In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go


import os
import sys
from BarG.Analysis import CoreAnalyzer as CA

def sep_meas(words):
    signs = '1234567890.'
    res = ''
    for item in words:
        if item in signs:
            res+=item
    return float(res)
    
def read_signal(way):
    file = open(way)
    file.readlines(13)
    res = pd.read_csv(file, header = None, sep = '\t', names = ['vol', 'sec'])
    file.close()
    return res


In [2]:
way = '/Users/lubimyj/Git/experiments/Plansee-Tungstan-W/'
material = ['Denal920','DX2']
specimen = ['Specimen1','Specimen2','Specimen3','Specimen4','Specimen5','Specimen6']
subFold = '/IRCamera/Specimen'
testFold = 'test/EXP #1/'
dataFold = '/FLT/'
files = ['Parameters.txt','incid.FLT','trans.FLT']
sr = 2e+9

In [4]:
shapes = pd.read_csv(way+material[1]+'/'+ material[1] + '.txt', sep = '\t')
shapes

,name,d,l
0,Specimen1,7.08,7.15
1,Specimen2,7.07,7.08
2,Specimen3,7.08,7.20
3,Specimen4,7.07,7.26
4,Specimen5,7.08,7.23
5,Specimen6,7.07,7.04


In [8]:

param = pd.read_csv(way + material[1]+'/'+ files[0],
                    header = None, names = ['title', 'value'], sep='\t')

#param['value'] = param['value'].apply(sep_meas)

#param.iloc[0,1] = float(shapes.d[0])*1e-3
#param.iloc[1,1] = float(shapes.l[0])*1e-3
#param.iloc[3,1] = param.value[3]*1000000000

#np_param = np.array(param.value)

param


,title,value
Bar Diameter,1.900000e-02,[m]
Young's Modulus,1.900000e+11,[Pa]
First Gauge,9.550000e-01,[m]
Second Gauge,5.200000e-01,[m]
Sound Velocity,4.800000e+03,[m/s]
Gauge Factor,2.120000e+00,[0]
Bridge Tension,1.018000e+01,[V]
Spacing,6.000000e+01,Points
Prominence,5.000000e+01,%
Curve Smoothing Parameter,7.100000e+03,[0]


In [7]:
path = way + material[1] + subFold + specimen[0] + testFold + dataFold
signal1 = read_signal(path +files[1])
signal2 = read_signal(path +files[2])

if max(signal1.vol) > max(signal2.vol):
    incid = signal1.vol-signal1.vol[0]
    transm = signal2.vol-signal2.vol[0]
else:
    incid = signal2.vol-signal2.vol[0]
    transm = signal1.vol-signal1.vol[0]
time = signal1.sec

data = pd.DataFrame({'time': time,
                    'incid': incid,
                    'trans': transm})
                    

In [8]:
ca = CA.CoreAnalyzer(path, np_param)
ca.load_experiments(np.array(incid), np.array(transm), np.array(time))

True

In [9]:
ca.analyze()


...detecting signal peaks with Sav - Gol filter with 57 window length (Curve Smoothing).

...Peaks detected in both signals. 

Automatic Cropping CMPLT.
...obtaining Fourier components
...Fourier components obtained, performing inverse Fourier transform...
Dispersion Correction CMPLT.

Signal cross - corelation Initialized...

Cross - Correlation CMPLT.
Final Calculation Commencing...
Unable to calculate Mean Striker Velocity.
Final Calculation CMPLT.


True

In [13]:
true_vectors = pd.DataFrame({'strain':ca.true_stress_strain[0],
                            'stress': ca.true_stress_strain[1]})
eng_vectors = pd.DataFrame({'strain':ca.eng_stress_strain[0],
                            'stress':  ca.eng_stress_strain[1]})

In [24]:
final_table = pd.DataFrame({'time': ca.time,
                            'incid': ca.corr_incid.y,
                            'trans': ca.corr_trans.y,
                            'refl': ca.corr_refle.y,
                            'U_in': ca.u_in,
                            'U_out': ca.u_out,
                            'eng_strain': ca.eng_stress_strain[0],
                            'eng_stress': ca.eng_stress_strain[1],
                            'true_strain': ca.true_stress_strain[0],
                            'true_stress': ca.true_stress_strain[1],
                            'F_in': ca.F_in,
                            'F_out': ca.F_out })
way_to_file = way + material[1] + subFold + specimen[0]
#final_table.to_csv(way_to_file + 'result_data.txt', index = False, sep = '\t')

In [15]:
final_table

,time,incid,trans,refl,U_in,U_out,eng_strain,eng_stress,true_strain,true_stress,F_in,F_out
0,5.000000e-07,-0.001063,-0.000216,0.000211,5.225017e-07,1.085495e-07,0.000020,548.444632,0.000020,548.433613,85070.523495,21591.816824
1,1.000000e-06,-0.000964,-0.000272,0.000112,9.147909e-07,2.104797e-07,0.000040,689.327716,0.000040,689.300168,84999.618016,27138.268662
2,1.500000e-06,-0.000481,-0.000186,0.000207,1.142483e-06,2.688843e-07,0.000076,472.965805,0.000076,472.929834,27323.908252,18620.277093
3,2.000000e-06,0.000037,-0.000076,0.000373,1.241588e-06,2.851516e-07,0.000126,193.013035,0.000126,192.988645,-40947.804710,7598.765409
4,2.500000e-06,0.000326,0.000003,0.000436,1.219766e-06,2.938308e-07,0.000172,7.519956,0.000172,7.518665,-76052.948177,-296.054504
...,...,...,...,...,...,...,...,...,...,...,...,...
348,1.745000e-04,-0.002048,-0.001027,0.000085,2.765066e-03,8.205535e-04,0.283810,2604.232647,0.333809,1865.126525,196011.644210,102526.510364
349,1.750000e-04,-0.001935,-0.000987,0.000062,2.765891e-03,8.209969e-04,0.283813,2503.279048,0.333813,1792.817094,187006.652498,98552.049701
350,1.755000e-04,-0.001726,-0.001007,-0.000015,2.766577e-03,8.214377e-04,0.283810,2553.258454,0.333809,1828.619178,173740.513880,100519.698034
351,1.760000e-04,-0.001406,-0.000975,-0.000032,2.767175e-03,8.219011e-04,0.283810,2472.504619,0.333809,1770.783728,143666.740159,97340.485568


In [25]:
res = pd.read_csv(way_to_file + 'result_data.txt', sep = '\t')

In [23]:
way_to_file

'/Users/lubimyj/Git/experiments/Plansee-Tungstan-W/Denal920/IRCamera/Specimen1/'

In [17]:
res

,time,incid,trans,refl,U_in,U_out,eng_strain,eng_stress,true_strain,true_stress,F_in,F_out
0,5.000000e-07,-0.001063,-0.000216,0.000211,5.225017e-07,1.085495e-07,0.000020,54.689863,0.000020,54.688752,8507.052349,2159.181682
1,1.000000e-06,-0.000964,-0.000272,0.000112,9.147909e-07,2.104797e-07,0.000040,68.738458,0.000040,68.735680,8499.961802,2713.826866
2,1.500000e-06,-0.000481,-0.000186,0.000207,1.142483e-06,2.688843e-07,0.000077,47.163257,0.000077,47.159629,2732.390825,1862.027709
3,2.000000e-06,0.000037,-0.000076,0.000373,1.241588e-06,2.851516e-07,0.000128,19.246895,0.000128,19.244436,-4094.780471,759.876541
4,2.500000e-06,0.000326,0.000003,0.000436,1.219766e-06,2.938308e-07,0.000174,0.749876,0.000174,0.749746,-7605.294818,-29.605450
...,...,...,...,...,...,...,...,...,...,...,...,...
348,1.745000e-04,-0.002048,-0.001027,0.000085,2.765066e-03,8.205535e-04,0.287021,259.689161,0.338303,185.152923,19601.164421,10252.651036
349,1.750000e-04,-0.001935,-0.000987,0.000062,2.765891e-03,8.209969e-04,0.287024,249.622259,0.338307,177.974684,18700.665250,9855.204970
350,1.755000e-04,-0.001726,-0.001007,-0.000015,2.766577e-03,8.214377e-04,0.287021,254.606111,0.338303,181.528803,17374.051388,10051.969803
351,1.760000e-04,-0.001406,-0.000975,-0.000032,2.767175e-03,8.219011e-04,0.287021,246.553491,0.338304,175.787421,14366.674016,9734.048557


In [27]:
fig1 = go.Figure()
fig1.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f"Material - {material[0]} Specimen №{specimen[0][0:-1]}")
fig1.add_trace(go.Scatter(x = data.time, y = data.incid, name = 'Incident'))
fig1.add_trace(go.Scatter(x = data.time, y = data.trans, name = 'Transmitted'))

fig1.update_yaxes(title = 'Voltage')
fig1.update_xaxes(title = 'Time')
fig1.show()

In [20]:
fig2 = go.Figure()
fig2.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f"Material - {material[0]} Specimen №{specimen[0][0:-1]}")
fig2.add_trace(go.Scatter(x = true_vectors.strain, y = true_vectors.stress, name = 'True'))
fig2.add_trace(go.Scatter(x = eng_vectors.strain, y = eng_vectors.stress, name = 'Engenering'))
fig2.update_yaxes(title = 'Voltage')
fig2.update_xaxes(title = 'Time')
fig2.show()

In [66]:
fig2 = go.Figure()
fig2.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f"Material - {material[0]} Specimen №{specimen[0][0:-1]}")
fig2.add_trace(go.Scatter(x = ca.time, y = ca.corr_incid.y, name = 'Incident'))
fig2.add_trace(go.Scatter(x = ca.time, y = ca.corr_trans.y, name = 'Transmited'))
fig2.add_trace(go.Scatter(x = ca.time, y = ca.corr_refle.y, name = 'Transmited'))
fig2.update_yaxes(title = 'Voltage')
fig2.update_xaxes(title = 'Time')
fig2.show()

In [31]:
fig2 = go.Figure()
fig2.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f"Material - {material[0]} Specimen №{specimen[0][0:-1]}")
fig2.add_trace(go.Scatter(x = time, y = ca.u_in, name = 'Incident'))
fig2.add_trace(go.Scatter(x = time, y = ca.u_out, name = 'Transmited'))
fig2.update_yaxes(title = 'Voltage')
fig2.update_xaxes(title = 'Time')
fig2.show()

In [26]:
fig2 = go.Figure()
fig2.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f"Material - {material[0]} Specimen №{specimen[0][0:-1]}")
fig2.add_trace(go.Scatter(x = res.true_strain, y = res.true_stress, name = 'Eng'))
#fig2.add_trace(go.Scatter(x = ca.true_stress_strain[0], y = ca.true_stress_strain[1], name = 'True'))
fig2.update_yaxes(title = 'Voltage')
fig2.update_xaxes(title = 'Time')
fig2.show()

In [33]:
fig2 = go.Figure()
fig2.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f"Material - {material[0]} Specimen №{specimen[0][0:-1]}")
fig2.add_trace(go.Scatter(x = time, y = ca.F_in, name = 'Eng'))
fig2.add_trace(go.Scatter(x = time, y = ca.F_out, name = 'True'))
fig2.update_yaxes(title = 'Voltage')
fig2.update_xaxes(title = 'Time')
fig2.show()